# 1D-CNN for ECG Arrhythmia Detection
## Complete Medical-Grade Pipeline using MIT-BIH Arrhythmia Database

This notebook implements a comprehensive deep learning solution for binary ECG arrhythmia classification following strict medical validation protocols.

## Section 1: Import Required Libraries

Install and import all necessary libraries for data processing, model building, and evaluation.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

## Section 2: Load MIT-BIH Arrhythmia Dataset

Load the training and test datasets from the Kaggle MIT-BIH ECG dataset.

In [ ]:
# Load the MIT-BIH Arrhythmia Database
# Train and test datasets from Kaggle
train_data = pd.read_csv('ecg_data/mitbih_train.csv', header=None)
test_data = pd.read_csv('ecg_data/mitbih_test.csv', header=None)

print(f"✓ Training data loaded: Shape = {train_data.shape}")
print(f"✓ Test data loaded: Shape = {test_data.shape}")
print(f"\nDataset Overview:")
print(f"  - Total ECG samples: {train_data.shape[0] + test_data.shape[0]}")
print(f"  - ECG signal length: {train_data.shape[1] - 1} time points")
print(f"  - Last column contains class labels (0-4)")

# Display first few rows
print(f"\nFirst 3 rows of training data:")
print(train_data.head(3))

## Section 3: Binary Label Conversion

Convert multi-class labels (0-4) to binary classification:
- **0** = Normal (No arrhythmia)
- **1-4** = Abnormal (Any type of arrhythmia)

## 2. Designing the 1D-CNN Architecture

In [ ]:
# Extract features and labels from training data
X_train_full = train_data.iloc[:, :-1].values  # All columns except last
y_train_full = train_data.iloc[:, -1].values   # Last column (labels)

# Extract features and labels from test data
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

# Convert multi-class labels to binary: 0 = Normal, 1-4 = Abnormal → 1
y_train_binary = (y_train_full > 0).astype(int)
y_test_binary = (y_test > 0).astype(int)

print("✓ Binary Label Conversion Complete")
print(f"\nLabel Distribution (Original Multi-class):")
print(f"  Training: {np.unique(y_train_full, return_counts=True)}")
print(f"  Test: {np.unique(y_test, return_counts=True)}")

print(f"\nLabel Distribution (Binary - 0=Normal, 1=Abnormal):")
unique_train, counts_train = np.unique(y_train_binary, return_counts=True)
unique_test, counts_test = np.unique(y_test_binary, return_counts=True)
print(f"  Training - Normal: {counts_train[0]}, Abnormal: {counts_train[1]}")
print(f"  Test - Normal: {counts_test[0]}, Abnormal: {counts_test[1]}")

## Section 5: Stratified Train-Validation Split

Perform stratified 80-20 split on training data BEFORE any resampling to prevent data leakage.
This maintains class distribution in both training and validation sets.

In [ ]:
# Reshape data to (Samples, 187, 1) for 1D-CNN
X_train_reshaped = X_train_full.reshape(-1, 187, 1)
X_test_reshaped = X_test.reshape(-1, 187, 1)

print("✓ Data Reshaping Complete for 1D-CNN")
print(f"\nOriginal shape: {X_train_full.shape}")
print(f"Reshaped shape: {X_train_reshaped.shape}")
print(f"\nTest data reshaped: {X_test_reshaped.shape}")
print(f"\nData is ready for 1D-CNN architecture")

## 4. Evaluation and Visualization

In [ ]:
# ⚠️ CRITICAL: Perform stratified split BEFORE resampling to prevent data leakage
X_train, X_val, y_train, y_val = train_test_split(
    X_train_reshaped, 
    y_train_binary, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train_binary
)

print("✓ Stratified Train-Validation Split Complete (80-20)")
print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test_reshaped.shape[0]} samples")

print(f"\nClass distribution preserved:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_val, counts_val = np.unique(y_val, return_counts=True)
print(f"  Training - Normal: {counts_train[0]}, Abnormal: {counts_train[1]}")
print(f"  Validation - Normal: {counts_val[0]}, Abnormal: {counts_val[1]}")
print(f"  Train ratio (Normal:Abnormal) = {counts_train[0]/counts_train[1]:.3f}")
print(f"  Val ratio (Normal:Abnormal) = {counts_val[0]/counts_val[1]:.3f}")

## Section 6: Visualize Class Imbalance (Before SMOTE)

Create a bar chart showing the class distribution of the training set BEFORE resampling.
This demonstrates the severe imbalance in the original data.

In [ ]:
# Visualize class imbalance BEFORE SMOTE
unique_before, counts_before = np.unique(y_train, return_counts=True)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(['Normal (0)', 'Abnormal (1)'], counts_before, color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=2)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}\n({height/len(y_train)*100:.1f}%)',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Number of Samples', fontsize=12, fontweight='bold')
ax.set_title('Class Distribution BEFORE SMOTE (Training Set)\nSevere Imbalance: ~90% Normal vs ~10% Abnormal', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_ylim(0, max(counts_before) * 1.15)
ax.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print(f"✓ Class Imbalance Visualization Complete")
print(f"\nImbalance Ratio: {counts_before[0]/counts_before[1]:.2f}:1 (Normal:Abnormal)")

## Section 7: Apply SMOTE to Training Set Only

Apply SMOTE (Synthetic Minority Over-sampling Technique) ONLY to the training set to achieve 1:1 class balance.
This prevents data leakage and ensures validation/test sets remain unmodified.

In [ ]:
# Apply SMOTE to training set only (NO RESAMPLING on validation/test)
# Flatten data for SMOTE
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote_flat, y_train_smote = smote.fit_resample(X_train_flat, y_train)

# Reshape back to 3D for CNN
X_train_smote = X_train_smote_flat.reshape(-1, 187, 1)

print("✓ SMOTE Applied to Training Set")
print(f"\nBefore SMOTE: {X_train.shape[0]} samples")
print(f"After SMOTE: {X_train_smote.shape[0]} samples")
print(f"\nClass distribution after SMOTE:")
unique_after, counts_after = np.unique(y_train_smote, return_counts=True)
print(f"  Normal (0): {counts_after[0]}")
print(f"  Abnormal (1): {counts_after[1]}")
print(f"  Ratio: 1:{counts_after[1]/counts_after[0]:.3f}")

## Section 8: Visualize Balanced Classes (After SMOTE)

Create a bar chart showing the perfectly balanced class distribution after SMOTE application.

In [ ]:
# Visualize balanced classes AFTER SMOTE
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Before SMOTE
axes[0].bar(['Normal (0)', 'Abnormal (1)'], counts_before, color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=2)
axes[0].set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
axes[0].set_title('BEFORE SMOTE\nImbalanced Data', fontsize=12, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
for i, (bar, count) in enumerate(zip(axes[0].patches, counts_before)):
    axes[0].text(bar.get_x() + bar.get_width()/2., count,
                f'{int(count)}\n({count/len(y_train)*100:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# After SMOTE
axes[1].bar(['Normal (0)', 'Abnormal (1)'], counts_after, color=['#2ecc71', '#e74c3c'], edgecolor='black', linewidth=2)
axes[1].set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
axes[1].set_title('AFTER SMOTE\nPerfectly Balanced (1:1 ratio)', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3, linestyle='--')
for i, (bar, count) in enumerate(zip(axes[1].patches, counts_after)):
    axes[1].text(bar.get_x() + bar.get_width()/2., count,
                f'{int(count)}\n({count/len(y_train_smote)*100:.1f}%)',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"✓ SMOTE Effectiveness Confirmed!")
print(f"\nImbalance Ratio Change:")
print(f"  Before: {counts_before[0]/counts_before[1]:.2f}:1")
print(f"  After: {counts_after[0]/counts_after[1]:.2f}:1")

## Section 9: Build 1D-CNN Architecture

Construct a 1D-CNN model with:
- 3 Conv1D layers (32 → 64 → 128 filters)
- BatchNormalization after each convolution
- MaxPooling1D for feature reduction
- Dropout(0.3) for regularization
- Dense output layer with sigmoid activation for binary classification

In [ ]:
# Build 1D-CNN model for ECG classification
model = models.Sequential([
    # First Conv Block
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(187, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    
    # Second Conv Block
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    
    # Third Conv Block
    layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    
    # Flatten and Dense
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    
    # Output layer for binary classification
    layers.Dense(1, activation='sigmoid')
])

# Display model architecture
model.summary()
print("\n✓ 1D-CNN Model Architecture Created Successfully")

## Section 10: Compile Model with Optimizer and Loss

Compile the model using:
- **Optimizer**: Adam (adaptive learning rate)
- **Loss**: binary_crossentropy (for binary classification)
- **Metrics**: Accuracy

In [ ]:
# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✓ Model Compiled Successfully")
print(f"\nOptimizer: Adam (lr=0.001)")
print(f"Loss Function: binary_crossentropy")
print(f"Metrics: accuracy")

## Section 11: Train Model with Early Stopping

Train the model on SMOTE-balanced training data with:
- Validation on the unseen validation set
- EarlyStopping callback to prevent overfitting (patience=5)
- 50 epochs with batch size 32

In [ ]:
# Define EarlyStopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model on SMOTE-balanced training data
print("🚀 Training the 1D-CNN model...\n")
history = model.fit(
    X_train_smote, y_train_smote,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

print("\n✓ Model Training Complete!")

## Section 12: Plot Training History (Accuracy & Loss)

Visualize training and validation accuracy and loss over epochs to assess model learning.

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, marker='o', markersize=4)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, marker='s', markersize=4)
axes[0].set_xlabel('Epoch', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=11, fontweight='bold')
axes[0].set_title('Model Accuracy vs Epochs', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, linestyle='--')

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2, marker='o', markersize=4)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, marker='s', markersize=4)
axes[1].set_xlabel('Epoch', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Loss', fontsize=11, fontweight='bold')
axes[1].set_title('Model Loss vs Epochs', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print(f"✓ Training history visualization complete")
print(f"\nFinal Training Results:")
print(f"  Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"  Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"  Training Loss: {history.history['loss'][-1]:.4f}")
print(f"  Validation Loss: {history.history['val_loss'][-1]:.4f}")

## Section 13: Evaluate on Test Set

Evaluate the trained model on the purely unseen MIT-BIH test set (mitbih_test.csv).

In [ ]:
# Evaluate on unseen test set
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test_binary, verbose=0)

print("=" * 60)
print("TEST SET EVALUATION (Unseen Data)")
print("=" * 60)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print("=" * 60)

## Section 14: Generate Classification Report

Generate a detailed classification report showing Precision, Recall, F1-Score for both Normal and Abnormal classes.

In [ ]:
# Generate predictions on test set
y_test_pred_proba = model.predict(X_test_reshaped)
y_test_pred = (y_test_pred_proba > 0.5).astype(int).flatten()

# Generate classification report
print("\n" + "=" * 70)
print("CLASSIFICATION REPORT (Test Set)")
print("=" * 70)
print(classification_report(
    y_test_binary, 
    y_test_pred,
    target_names=['Normal (0)', 'Abnormal (1)'],
    digits=4
))
print("=" * 70)

## Section 15: Plot Confusion Matrix Heatmap

Visualize True Positives, True Negatives, False Positives, and False Negatives.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test_binary, y_test_pred)

# Plot confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Normal (0)', 'Abnormal (1)'],
            yticklabels=['Normal (0)', 'Abnormal (1)'],
            annot_kws={'size': 14, 'weight': 'bold'})
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - ECG Arrhythmia Detection (Test Set)', 
          fontsize=13, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

# Print confusion matrix analysis
tn, fp, fn, tp = cm.ravel()
print("\n" + "=" * 70)
print("CONFUSION MATRIX ANALYSIS")
print("=" * 70)
print(f"True Negatives (Normal correctly identified):   {tn}")
print(f"False Positives (Normal misclassified):         {fp}")
print(f"False Negatives (Abnormal misclassified):       {fn}")
print(f"True Positives (Abnormal correctly identified): {tp}")
print("=" * 70)
print(f"\nSensitivity (Recall for Abnormal): {tp/(tp+fn):.4f}")
print(f"Specificity (Recall for Normal):   {tn/(tn+fp):.4f}")
print("=" * 70)

## Section 16: Save Trained Model

Save the final trained model to disk for future inference and deployment.

In [ ]:
# Save the trained model
model_save_path = 'ecg_model_final.h5'
model.save(model_save_path)

print(f"✓ Model saved successfully!")
print(f"  Location: {model_save_path}")
print(f"  File size: {os.path.getsize(model_save_path) / (1024*1024):.2f} MB")

# Verify model can be loaded
loaded_model = keras.models.load_model(model_save_path)
print(f"\n✓ Model verification: Successfully loaded from disk")
print(f"  Model can be used for inference!")

print("\n" + "=" * 70)
print("TRAINING PIPELINE COMPLETE")
print("=" * 70)
print(f"""
✓ All steps completed successfully:
  1. ✓ Data loaded from MIT-BIH Arrhythmia Database
  2. ✓ Binary classification (Normal vs Abnormal)
  3. ✓ Data reshaped for 1D-CNN
  4. ✓ Stratified train-validation split (80-20)
  5. ✓ Class imbalance visualized
  6. ✓ SMOTE applied to training set only
  7. ✓ Balanced classes verified
  8. ✓ 1D-CNN architecture built
  9. ✓ Model compiled (Adam + binary_crossentropy)
  10. ✓ Model trained with EarlyStopping
  11. ✓ Training history plotted
  12. ✓ Evaluated on unseen test set
  13. ✓ Classification report generated
  14. ✓ Confusion matrix visualized
  15. ✓ Model saved as 'ecg_model_final.h5'
  
Final Test Accuracy: {test_accuracy*100:.2f}%
Ready for deployment!
""")
print("=" * 70)